In [ ]:
from __init__ import *

In [ ]:
directory_path = "{}mutation/".format(SETTING["directory_path"])

kraft.path.make(directory_path)

In [ ]:
for url in ("https://ndownloader.figshare.com/files/22629110",):

    kraft.internet.download(
        url, directory_path, name="CCLE_mutations.csv", overwrite=SETTING["overwrite"]
    )

In [ ]:
_2d_array = pd.read_csv(
    "{}CCLE_mutations.csv".format(directory_path),
    sep="\t",
    usecols=(0, 7, 18, 32, 33),
).to_numpy()

len(_2d_array)

In [ ]:
_2d_array = _2d_array[_2d_array[:, 3] == "damaging", :]

len(_2d_array)

In [ ]:
gene_, variant_classification_, protein_change_, sample_ = _2d_array[:, (0, 1, 2, 4)].T

In [ ]:
sample_ = kraft.name_biology.name_cell_line(sample_)

sample_to_index = kraft.array.map_int(sample_)[0]

len(sample_to_index)

In [ ]:
gene_to_index = kraft.array.map_int(gene_)[0]

len(gene_to_index)

In [ ]:
def combine(gene, feature):

    if not isinstance(feature, str):

        feature = ""

    return "{}_{}".format(gene, feature)

In [ ]:
variant_classification_ = np.asarray(
    tuple(
        combine(gene, variant_classification)
        for gene, variant_classification in zip(gene_, variant_classification_)
    )
)

variant_classification_to_index = kraft.array.map_int(variant_classification_)[0]

len(variant_classification_to_index)

In [ ]:
protein_change_ = np.asarray(
    tuple(
        combine(gene, protein_change)
        for gene, protein_change in zip(gene_, protein_change_)
    )
)

protein_change_to_index = kraft.array.map_int(protein_change_)[0]

len(protein_change_to_index)

In [ ]:
sample_n = len(sample_to_index)

gene_x_sample = np.full((len(gene_to_index), sample_n), 0)

variant_classification_x_sample = np.full(
    (len(variant_classification_to_index), sample_n), 0
)

protein_change_x_sample = np.full((len(protein_change_to_index), sample_n), 0)

In [ ]:
kraft.dict.summarize(sample_to_index)

In [ ]:
for gene, variant_classification, protein_change, sample in zip(
    gene_, variant_classification_, protein_change_, sample_
):

    sample_index = sample_to_index[sample]

    gene_x_sample[gene_to_index[gene], sample_index] += 1

    if not isinstance(variant_classification, str):

        variant_classification_ = ""

    variant_classification_x_sample[
        variant_classification_to_index[variant_classification],
        sample_index,
    ] += 1

    if not isinstance(protein_change, str):

        protein_change_ = ""

    protein_change_x_sample[protein_change_to_index[protein_change], sample_index] += 1

In [ ]:
columns = pd.Index(data=sample_to_index, name="Cell Line")

In [ ]:
gene_x_sample = pd.DataFrame(data=gene_x_sample, index=gene_to_index, columns=columns)

gene_x_sample.index.name = "Gene"

gene_x_sample.to_csv("{}gene_x_sample.tsv".format(directory_path), sep="\t")

gene_x_sample

In [ ]:
gene_x_sample_x_01 = gene_x_sample.clip(upper=1)

gene_x_sample_x_01.to_csv("{}gene_x_sample_x_01.tsv".format(directory_path), sep="\t")

gene_x_sample_x_01

In [ ]:
variant_classification_x_sample = pd.DataFrame(
    data=variant_classification_x_sample,
    index=variant_classification_to_index,
    columns=columns,
)

variant_classification_x_sample.index.name = "Gene Variant Classification"

variant_classification_x_sample.to_csv(
    "{}variant_classification_x_sample.tsv".format(directory_path), sep="\t"
)

variant_classification_x_sample

In [ ]:
protein_change_x_sample = pd.DataFrame(
    data=protein_change_x_sample,
    index=protein_change_to_index,
    columns=columns,
)

protein_change_x_sample.index.name = "Protein Change"

protein_change_x_sample.to_csv(
    "{}protein_change_x_sample.tsv".format(directory_path), sep="\t"
)

protein_change_x_sample